In [1]:
from playwright.async_api import async_playwright, Page

In [2]:
playwright = await async_playwright().start()
download_path = "downloads"
browser = await playwright.chromium.launch(headless=False,args=[
        '--disable-pdf-viewer',
        '--plugins.always_open_pdf_externally=true',
        f'--default-download-directory={download_path}'
    ] )
context = await browser.new_context(accept_downloads=True)        
page = await context.new_page()
await page.goto("https://wisetechacademy.com/log-in")

<Response url='https://wisetechacademy.com/log-in' request=<Request url='https://wisetechacademy.com/log-in' method='GET'>>

In [3]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.ERROR,  # Set to DEBUG for more detailed logs
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("chatgpt_logs.log"),  # Log to a file
        logging.StreamHandler()  # Log to the console
    ]
)
logger = logging.getLogger("ChatGPTLogger")

In [4]:
import time
import csv
page.set_default_timeout(5000)
# Loop through each link and click it one by one
with open('output.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # writer.writerow(['no', 'header','text']) 
    count = 1277
    
    while True:
        # Wait for the pagination container to be visible
        await page.wait_for_selector('div.search-pagination')

        # Extract current page number and print it
    
        current_page = await page.locator('a.search-pagination-page.current-page').inner_text()
        print(f"Processing Page: {current_page}")

        # Perform your data scraping or processing logic here
        # Example: Extract text or data from the current page
        page_content = await page.content()  # Modify this line as needed to extract data
        print(f"Page {current_page} content extracted")

        # Check if the next button is enabled
        await page.wait_for_selector('a.search-pagination-button.enabled-link')
        
        next_buttons = page.locator('a.search-pagination-button.enabled-link')
        if await next_buttons.count() == 0:
            print("No more pages to navigate.")
            break

        # Click on the last "next" button (use index -1 for the last element)
        

        await page.wait_for_selector('a.search-result-title')
        links = await page.query_selector_all("a.search-result-title")
        print(links)
        for link in links:
            try:
                header = None
                await link.click()
                await page.wait_for_selector('h2.modal-title span')
                header = await page.locator('h2.modal-title span').text_content()
                # await page.wait_for_selector('h3[data-exp-question]')
                                            
                # await page.locator('h3[data-exp-question]').click()
                # await page.wait_for_selector('div[data-exp-answer]')
                
                
                text = await page.locator('div[data-exp-answer]').text_content()
                close_button = await page.query_selector('button.btn-close[data-bs-dismiss="modal"]')
                await close_button.click()
                writer.writerow([str(count), header,  text])
                count+=1
            except Exception as e:
                if header:
                    logger.error(header)
                else:
                    print(e)
        # Click the next button and wait for the new page to load
        await next_buttons.nth(-1).click()
        await page.wait_for_load_state('networkidle')
            


Processing Page: 8
Page 8 content extracted
[<JSHandle preview=JSHandle@<a data-v-55f30f84="" data-bs-toggle="tooltip" class="search-result-title card-heading-color-visited smaller" title="What do the visual board buttons and indicators in the top-right pane mean? ">What do the visual board buttons and indicators i…</a>>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>]
Processing Page: 9
Page 9 cont

CancelledError: 

In [48]:
links = await page.query_selector_all("a.search-result-title")
print(links)

[<JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>]


In [48]:
# await links[0].click(force=True)

In [26]:
import csv
async def save_to_csv(page, writer):
    modal_text = await page.inner_text('.modal-body')
    pdf_links = await page.eval_on_selector_all('a[href$=".pdf"]', 'elements => elements.map(el => el.href)')

    # Optional: Split text into sections if necessary, e.g., for better organization

    # Write the extracted data to a CSV file
    writer.writerow([modal_text.strip(), list(set(pdf_links))])

OperationalError: unable to open database file

In [5]:
import json

# read HAR file
with open("wisetechacademy.com.har", "r") as file:
    har_data = json.load(file)

# extract Cookies information
cookies = har_data["log"]["entries"][0]["response"]["cookies"]

# print out Cookies
for cookie in cookies:
    print(cookie["name"], cookie["value"])